In [1]:
from pair import CarbonPair, __version__ as pair_v, __date__ as pair_d
print("CarbonPair version", pair_v, pair_d)

CarbonPair version 1.2.1 13/Nov/2022


# Carbon Pair

The `CarbonPair` class contains static information about a pair. Most important is the price convention, ie in which directions prices are quoted. It also has some price related functionality, including the `price` function that quotes prices in the correct convention, and the `convert` function that converts token amounts taking the price convention into account

In [2]:
help(CarbonPair)

Help on class CarbonPair in module pair:

class CarbonPair(builtins.object)
 |  CarbonPair(tknb: str, tknq: str) -> None
 |  
 |  static information about a carbon token pair
 |  
 |  :tknb:      the base token (risk token) of the pair, eg ETH*
 |  :tknq:      the quote token (numeraire token) of the pair, eg USDC*
 |  
 |  * the differentiation between numeraire and risk tokens matter only for price quotes:
 |  in a given pair, _all_ prices will be quote as tknn per tknr, eg USDC per ETH
 |  
 |  see also https://www.investopedia.com/terms/i/isocurrencycode.asp for ISO currency code
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |  
 |  __init__(self, tknb: str, tknq: str) -> None
 |  
 |  __post_init__(self)
 |  
 |  __repr__(self)
 |  
 |  convert(self, amtfrom, tknfrom, tknto, price)
 |      converts one token amount into the other, with price according to curve convention
 |      
 |      :amtfrom:   the amount of tknfrom
 |      :tknfrom:   source token
 |      :tkn

In [3]:
ETHUSDC = CarbonPair("ETH", "USDC")
ETHUSDC

CarbonPair(tknb='ETH', tknq='USDC')

the `reverse` property creates a new CarbonPair with the reverse price conventions

In [4]:
USDCETH = ETHUSDC.reverse
USDCETH

CarbonPair(tknb='USDC', tknq='ETH')

Get base token and quote token

In [5]:
ETHUSDC.tknb, ETHUSDC.tknq

('ETH', 'USDC')

We can also get a pair object from an iso string of length 6 or 8

In [6]:
ETHBNT = CarbonPair.from_isopair_and_tkn("ETHBNT")
ETHBNT.tknb, ETHBNT.tknq

('ETH', 'BNT')

In [7]:
AAVELINK = CarbonPair.from_isopair_and_tkn("AAVELINK")
AAVELINK.tknb, AAVELINK.tknq

('AAVE', 'LINK')

And we can also get it from the pair and one of the tokens in it (does not matter which one)

In [8]:
ETHUSDT = CarbonPair.from_isopair_and_tkn("ETHUSDT", "ETH")
ETHUSDT.tknb, ETHUSDT.tknq

('ETH', 'USDT')

In [9]:
ETHUSDT = CarbonPair.from_isopair_and_tkn("ETHUSDT", "USDT")
ETHUSDT.tknb, ETHUSDT.tknq

('ETH', 'USDT')

- `pair_id` is the canonic Carbon pair id which alphabetically orders the tokens and separates them with a colon; it does not convey any quote information

- `pair_iso` is the pair in iso convention, ie BASQUO; it does convey quote information, in that price units are "QUO per BAS"

- `pair_slash` is the pair in slash convention, ie BAS/QUO; it is equivalent to `pair_iso`

- `price_convention` is the price convention of the pair in human readable terms, ie "QUO per BAS"


In [10]:
ETHUSDC.pair_id, ETHUSDC.pair_iso, ETHUSDC.pair_slash, ETHUSDC.price_convention

('ETH:USDC', 'ETHUSDC', 'ETH/USDC', 'USDC per ETH')

for the reverse token all the above are reversed _except_ that canonic `pair_id`

In [11]:
USDCETH.pair_id, USDCETH.pair_iso, USDCETH.pair_slash, USDCETH.price_convention

('ETH:USDC', 'USDCETH', 'USDC/ETH', 'ETH per USDC')

`pair_id_is_reversed` is True iff and order in the canonic pair id is the opposite of that in the ISO pair name

In [12]:
ETHUSDC.pair_id, ETHUSDC.pair_iso, ETHUSDC.pair_id_is_reversed

('ETH:USDC', 'ETHUSDC', False)

In [13]:
USDCETH.pair_id, USDCETH.pair_iso, USDCETH.pair_id_is_reversed

('ETH:USDC', 'USDCETH', True)

here we check whether a specific token is the base token of a pair

In [14]:
ETHUSDC.has_basetoken("ETH"), ETHUSDC.has_basetoken("USDC"), ETHUSDC.has_basetoken("LINK")

(True, False, False)

ditto quote token

In [15]:
ETHUSDC.has_quotetoken("ETH"), ETHUSDC.has_quotetoken("USDC"), ETHUSDC.has_quotetoken("LINK")

(False, True, False)

ditto whether the token is part of the pair as either quote or base token

In [16]:
ETHUSDC.has_token("ETH"), ETHUSDC.has_token("USDC"), ETHUSDC.has_token("LINK")

(True, True, False)

returns the other token of the pair (None if not in pair)

In [17]:
ETHUSDC.other("ETH"), ETHUSDC.other("USDC"), ETHUSDC.other("LINK")

('USDC', 'ETH', None)

## Price-related functions

`price` converts a `price0` given as `tknq0` per `tknb0` into the correct units for the token pair (ie it is inversed if need be); if one of the tokens is not in the pair it returns None

In [18]:
help(ETHUSDC.price)

Help on method price in module pair:

price(price0, tknb0, tknq0) method of pair.CarbonPair instance
    the normalized price, according to convention
    
    :price0:    the raw price, as tknq0 per tknb0
    :tknb0:     the base token of price0
    :tknq0:     the quote token of price0
    :returns:   the amount of tarket token to be returned



In [19]:
ETHUSDC.price(2000, "ETH", "USDC"), ETHUSDC.price(0.0005, "USDC", "ETH"), ETHUSDC.price(1, "ETH", "LINK")

(2000, 2000.0, None)

`convert` converts the currency amount `amtfrom` in `tknfrom` to a corresponding amount in `tknto` at a price `price` that is quoted in the convention of the pair

In [20]:
help(ETHUSDC.convert)

Help on method convert in module pair:

convert(amtfrom, tknfrom, tknto, price) method of pair.CarbonPair instance
    converts one token amount into the other, with price according to curve convention
    
    :amtfrom:   the amount of tknfrom
    :tknfrom:   source token
    :tknto:     target token; can be the same as source token, in which case price is unity
    :price:     the price, in the price_convention of the pair
    :returns:   the amount of target token to be returned, or None if one of the tokens not in pair



In [21]:
ETHUSDC.convert(2, "ETH", "USDC", 2000), ETHUSDC.convert(4000, "USDC", "ETH", 2000)

(4000, 2.0)

`convert` reacts gracefully to `tknfrom == tknto` in which case the price is ignored and the exchange rate is unity

In [22]:
ETHUSDC.convert(2, "ETH", "ETH", 2000), ETHUSDC.convert(4000, "USDC", "USDC", 2000)

(2, 4000)

`convert_price` converts a price rather than an amount. It takes a `price` number, and the token `tknq` in which the price is expressed. The price is then returned in the price convention of the pair. For example, if 1 ETH = 2000 USDC then we can express this as `(price=2000, tknq='USDC')` or `(price=0.0005, tknq='ETH')`. 

In [23]:
help(ETHUSDC.convert_price)

Help on method convert_price in module pair:

convert_price(price, tknq) method of pair.CarbonPair instance
    converts a price expressed in the tknq numeraire in the numeraire of the pair
    
    :price:     the price to be converted, with numeraire being `tknq`
    :tknq:      the numeraire token in which the `price` is expressed
    :returns:   the price expressed in the numeraire conventions of the pair; 
                returns None if tknq not part of this pair



In [24]:
ETHUSDC.convert_price(2000, "USDC"), ETHUSDC.convert_price(0.0005, "ETH")

(2000, 2000.0)

`limit_is_met` checks whether a limit order is met. Both the `limit_price` and the `current_price` are provided in the price conventions of the pair. `buysell` indicates whether the person having PLACED the order is buying or selling `tkn`. The function will return True iff it is in the interest of that person to execute the trade. For example (assuming the pair `ETHUSDC`)

- `BUY`ing ETH at `current_price=2000` when `limit_price=1000` USD per ETH yields `False` (won't buy ETH above 1000)
- `SELL`ing ETH at `current_price=2000` when `limit_price=1000` USD per ETH yields `True` (will sell ETH above 1000)
- `BUY`ing USDC at `current_price=2000` when `limit_price=1000` USD per ETH yields `True` (will buy USD, aka sell ETH, above 1000)
- `SELL`ing USDC at `current_price=2000` when `limit_price=1000` USD per ETH yields `False` (won't sell USD, aka buy ETH, above 1000)

In [25]:
help(ETHUSDC.limit_is_met)

Help on method limit_is_met in module pair:

limit_is_met(tkn, limit_price, buysell, current_price, reverse=False, asphrase=False) method of pair.CarbonPair instance
    checks whether a limit order has been met
    
    :tkn:               the token to be bought or sold
    :limit_price:       the price at which the token is to be bought or sold; quoted
                        in the price convention of this pair
    :buysell:           whether the limit order is for buying or selling `tkn`; this is
                        from the point of view of the person who PLACED the order; use
                        `reverse=True` to do it from the point of view of the person
                        who ACCEPTED the orders; use the constants BUY and SELL
    :current_price:     the price which is tested against the limit order
    :asphrase:          returns the result as an explanatory phrase instead of bool 
    :returns:           True if the limit order will lead to execution, false if no

In [26]:
print(ETHUSDC.limit_is_met("ETH", 2000, ETHUSDC.BUY, 1000, asphrase=True))
ETHUSDC.limit_is_met("ETH", 2000, ETHUSDC.BUY, 1000)

Order placer buys ETH at 1000 USDC per ETH (limit=2000)


True

In [27]:
print(ETHUSDC.limit_is_met("ETH", 2000, ETHUSDC.SELL, 1000, asphrase=True))
ETHUSDC.limit_is_met("ETH", 2000, ETHUSDC.SELL, 1000)

Order placer does not sell ETH at 1000 USDC per ETH (limit=2000)


False

In [31]:
print(ETHUSDC.limit_is_met("USDC", 2000, ETHUSDC.BUY, 1000, asphrase=True))
ETHUSDC.limit_is_met("USDC", 2000, ETHUSDC.BUY, 1000), ETHUSDC.limit_is_met("ETH", 2000, ETHUSDC.SELL, 1000)

Order placer does not buy USDC at 1000 USDC per ETH (limit=2000)


(False, False)

In [32]:
print(ETHUSDC.limit_is_met("USDC", 2000, ETHUSDC.SELL, 1000, asphrase=True))
ETHUSDC.limit_is_met("USDC", 2000, ETHUSDC.SELL, 1000), ETHUSDC.limit_is_met("ETH", 2000, ETHUSDC.BUY, 1000)

Order placer sells USDC at 1000 USDC per ETH (limit=2000)


(True, True)

In [30]:
ETHUSDC.limit_is_met("ABCD", 2000, ETHUSDC.SELL, 1000),

[limit_is_met] invalid token ABCD for pair ETHUSDC


(None,)